# How to discover object types

For more, check out these links: 
- [Get-Member](https://docs.microsoft.com/en-us/powershell/module/microsoft.powershell.utility/get-member?view=powershell-7.3&WT.mc_id=ps-gethelp) and [about_Objects](https://docs.microsoft.com/en-us/powershell/module/microsoft.powershell.core/about/about_objects?view=powershell-7.3)
- And [Find-Member](https://github.com/SeeminglyScience/ClassExplorer/blob/master/docs/en-US/Find-Member.md) which is from one of my favorite modules:  [ClassExplorer](https://github.com/SeeminglyScience/ClassExplorer)
- [`Get-HelpFromType( $obj )`](https://github.com/ninmonkey/Mini.Examples-PowerShell/blob/main/Types/Get-HelpFromType.ps1)


## First Method : `GetType()`

In [ ]:
$result = 10 / 3.14
$result.GetType()


IsPublic IsSerial Name                                     BaseType
-------- -------- ----                                     --------
True     True     Double                                   System.ValueType



In [ ]:
  # Pwsh gives you sugar to cache values, like this
$Files ??= Get-ChildItem c:\ -depth 2 -ea ignore
$Files.count | Join-String -op 'Total Items: ' -form '{0:n0}'

# create some example objects to test
$SomeDir = Get-Item -Path .
$SomeFile = Get-ChildItem -LiteralPath c:\ -File | Select-Object -First 1 
$SomePS = Get-Process | Select -First 1

Total Items: 31,892


Notice that the type of `$files` is `[object[]]`
The first element is [DirectoryInfo]

In [ ]:

@(
    $SomeDir.GetType()
    $SomeFile.GetType()
    $SomePS.GetType()
    $Files.GetType()
    $files[0].GetType()
) | Format-Table -autosize -Wrap 



IsPublic IsSerial Name          BaseType
-------- -------- ----          --------
True     False    DirectoryInfo System.IO.FileSystemInfo
True     False    FileInfo      System.IO.FileSystemInfo
True     False    Process       System.ComponentModel.Component
True     True     Object[]      System.Array
True     False    DirectoryInfo System.IO.FileSystemInfo



`Object[]` means that it is an array of `[Object]` 
We need to index it to test the array's elements type

Here's a couple of Type information's properties

In [ ]:
$Files[0].GetType().FullName

System.IO.DirectoryInfo


In [ ]:
# Tip, these will never throw an error, no matter the type
@( $files )[0].GetType().Name
$x = $null
@( $x )[0]


DirectoryInfo


In [ ]:
$file = Get-Item '.'
$File.GetType().FullName

$File.PSTypeNames -join ', '

$File.PSTypeNames 
| formatShortType
| Join-String @splatCsvType

$File.PSTypeNames 
| FormatShortType
| join-String @splatUL


System.IO.DirectoryInfo
System.IO.DirectoryInfo, System.IO.FileSystemInfo, System.MarshalByRefObject, System.Object
formatShortType: 
Line |
   7 |  | formatShortType
     |    ~~~~~~~~~~~~~~~
     | The term 'formatShortType' is not recognized as a name of a cmdlet, function, script file, or executable program.
Check the spelling of the name, or if a path was included, verify that the path is correct and try again.
FormatShortType: 
Line |
  11 |  | FormatShortType
     |    ~~~~~~~~~~~~~~~
     | The term 'FormatShortType' is not recognized as a name of a cmdlet, function, script file, or executable program.
Check the spelling of the name, or if a path was included, verify that the path is correct and try again.


In [ ]:
# but calling a method on null will still Error
@( $x )[0].ToString()
@( $null )[0].ToString()

InvalidOperation: 
Line |
   2 |  @( $x )[0].ToString()
     |  ~~~~~~~~~~~~~~~~~~~~~
     | You cannot call a method on a null-valued expression.
InvalidOperation: 
Line |
   3 |  @( $null )[0].ToString()
     |  ~~~~~~~~~~~~~~~~~~~~~~~~
     | You cannot call a method on a null-valued expression.


In [ ]:
# Helper functions

function formatShortType { 
    # tiny sugar to abbreviate names
    param( [Parameter(Mandatory, ValueFromPipeline)]$InputObject )
    process { 
        if ($null -eq $InputObject) {
            "[`u{2400}]"
        } # null str symbol
        if ($InputObject -is 'type') {
            $ShortName = $InputObject.Name
        } elseif ($InputObject -is 'string') {
            $maybeTinfo = $InputObject -as 'type'
            if ($MaybeTinfo) {
                $ShortName = $maybeTinfo.Name 
            } else {
                $shortName = [string]$InputObject
            }
        } else {
            $tinfo = $InputObject.GetType()
            $ShortName = $Tinfo.Name
        }
        $ShortName -replace '^(System|System\.Management\.)', ''
    }
}



In [ ]:
$splatUL = @{
    Separator = "`n - "
    op = "`n - " ; Os = "`n"
    Property = { '[{0}]' -f $_ }
}
$splatCSVType = @{
    Sep = ', ' 
    # DoubleQuote = $True
    Property = { '[{0}]' -f $_ }
}
# $someDir.PsTypeNames | 
$someDir.PsTypeNames -join ', ' 
$someDir.PsTypeNames | FormatShortType | Join-String -sep ', ' -double { "[${_}]"}
$someDir.PsTypeNames | FormatShortType | Join-String @splatCsvType

System.IO.DirectoryInfo, System.IO.FileSystemInfo, System.MarshalByRefObject, System.Object
"[DirectoryInfo]", "[FileSystemInfo]", "[MarshalByRefObject]", "[Object]"
[DirectoryInfo], [FileSystemInfo], [MarshalByRefObject], [Object]


In [ ]:
# often you 
$SomeDir.GetType().Name
$SomeDir.GetType().Namespace
$SomeDir.GetType().FullName
$SomeDir.GetType().BaseType.FullName

DirectoryInfo
System.IO
System.IO.DirectoryInfo
System.IO.FileSystemInfo


Useful when you want a list of types, without duplicates. It uses type definitions -- verses default comparisons 

In [ ]:
$stuff = @(
    0..10
    Get-Item .
    PS | select -first 1
)
$Stuff.count
$Stuff | Get-Unique -OnType  
| formatSHortType | Join-String @splatCsvType
# $fileListTypes | Join-string @splatCsvType

13
[Int32], [DirectoryInfo], [Process]


In [ ]:
$someDir = Get-Item .
$someDir.PsTypeNames
| formatShortType | Sort-Object
| Join-string @splatUL
| Join-string -op "A folder = "


$somePS.PSTypeNames
| formatShortType | Sort-Object
| Join-string @splatUL
| Join-string -op "A process = "


A folder = 
 - [DirectoryInfo]
 - [FileSystemInfo]
 - [MarshalByRefObject]
 - [Object]

A process = 
 - [Component]
 - [MarshalByRefObject]
 - [Object]
 - [Process]



In [ ]:
function Get-HelpFromType {
    <#
    .synopsis
        open Powershell docs from a type name
    .description
        compied from: <https://github.com/ninmonkey/Mini.Examples-PowerShell/blob/main/Types/Get-HelpFromType.ps1>
    .example
        PS> HelpFromType int64
    .example
        PS> [System.Management.Automation.ErrorRecord] | Get-HelpFromType
    .example
        PS> (Get-Command ls) | HelpFromType
        # Loads docs on [AliasInfo]
        # <https://docs.microsoft.com/en-us/dotnet/api/System.Management.Automation.AliasInfo?view=powershellsdk-7.0.0>
    #>
    [Alias('TypeHelp', 'HelpFromType')]
    param(
        # object or type instance, should auto coerce to FullName
        [Parameter(Mandatory, Position = 0, ValueFromPipeline)]
        [object]$InputObject,

        # Return urls instead of opening browser pages
        [Parameter()][switch]$PassThru
    )

    process {
        if ($InputObject -is [string]) {
            $typeInstance = $InputObject -as [type]
            if ($null -eq $typeInstance) {
                Write-Debug "String, was not a type name: '$InputObject'"
                $typeName = 'System.String'
            }
            else {
                $typeName = $typeInstance.FullName
            }
        }
        elseif ( $InputObject -is [type] ) {
            $typeName = $InputObject.FullName
        }
        else {
            $typeName = $InputObject.GetType().FullName
        }
        $url = 'https://docs.microsoft.com/en-us/dotnet/api/{0}' -f $typeName

        if ($PassThru) {
            $url
            return
        }
        Start-Process $url
    }
}


In [ ]:
@( $SomeDir; $SomeFile; @( PS )[0]  )
| Get-HelpFromType -PassThru

https://docs.microsoft.com/en-us/dotnet/api/System.IO.DirectoryInfo
https://docs.microsoft.com/en-us/dotnet/api/System.IO.FileInfo
https://docs.microsoft.com/en-us/dotnet/api/System.Diagnostics.Process
